# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
import random
from sklearn.metrics import accuracy_score

import warnings
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Вывод**

Предобработка данных не требуется, можем начинать анализ. 

## Разбейте данные на выборки

Разделим данные в следующей пропорции
- Обучающая выборка - 60%
- Валидационная выборка - 20%
- Тестовая выборка - 20%

In [5]:
df_train, df_valid_test = train_test_split(data, test_size=0.4, random_state=12345, stratify = data['is_ultra']) 
df_valid, df_test = train_test_split(df_valid_test,test_size=0.5, random_state=12345, stratify = df_valid_test['is_ultra'])

## Исследуйте модели

Сохраним признаки в отдельных переменных

In [6]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

In [7]:
print(features_train.shape,target_train.shape)
print(features_valid.shape,target_valid.shape)
print(features_test.shape,target_test.shape)

(1928, 4) (1928,)
(643, 4) (643,)
(643, 4) (643,)


- ### Модель решающего дерева

In [8]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) # обучим модель
    predictions = model.predict(features_valid) # получим предсказания модели
    result = accuracy_score(target_valid, predictions) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        
model_Decision_Tree = DecisionTreeClassifier(random_state=12345, max_depth=best_depth)
model_Decision_Tree.fit(features_train, target_train)
        
print("Accuracy лучшей модели решающего дерева:", best_result)
print("Глубина дерева", best_depth)

Accuracy лучшей модели решающего дерева: 0.7853810264385692
Глубина дерева 5


- ### Модель случайный лес

In [9]:
best_model = None
best_result = 0
best_n_estimator = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model
        best_result = result
        best_n_estimator = est

model_Random_Forest = RandomForestClassifier(random_state=12345, n_estimators=best_n_estimator)
model_Random_Forest.fit(features_train, target_train)

print("Accuracy лучшей модели случайного леса:", best_result)
print("n_estimators лучшей модели", best_n_estimator)

Accuracy лучшей модели случайного леса: 0.7869362363919129
n_estimators лучшей модели 10


In [10]:
best_model = None
best_result = 0
best_n_estimator = 0
best_depth = 0
for est in range(1, 11):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth) 
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
        if result > best_result:
            best_model = model
            best_result = result
            best_n_estimator = est
            best_depth = depth

model_Random_Forest = RandomForestClassifier(random_state=12345, n_estimators=best_n_estimator, max_depth = depth)
model_Random_Forest.fit(features_train, target_train)

print("Accuracy лучшей модели случайного леса:", best_result)
print("n_estimators лучшей модели", best_n_estimator)
print("max_depth лучшей модели", best_depth)

Accuracy лучшей модели случайного леса: 0.7962674961119751
n_estimators лучшей модели 7
max_depth лучшей модели 3


- ### Модель логистической регрессии

In [11]:
model_Log_Regression = LogisticRegression(random_state=12345)
model_Log_Regression.fit(features_train, target_train)
result = model_Log_Regression.score(features_valid, target_valid) 
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.71850699844479


## Проверьте модель на тестовой выборке

In [12]:
predictions = model_Decision_Tree.predict(features_test)
print("Accuracy лучшей модели решающего дерева:", accuracy_score(target_test, predictions)) 

predictions = model_Random_Forest.predict(features_test)
print("Accuracy лучшей модели случайного леса:", accuracy_score(target_test, predictions))

predictions = model_Log_Regression.predict(features_test)
print("Accuracy модели логистической регрессии:", accuracy_score(target_test, predictions))

Accuracy лучшей модели решающего дерева: 0.8118195956454122
Accuracy лучшей модели случайного леса: 0.8055987558320373
Accuracy модели логистической регрессии: 0.7107309486780715


**Вывод**

На тестовой выборке наилучшее Accuracy получилось у модели решающего дерева - 0,81

## Проверьте модели на адекватность

Проверим модель на адекватность с помощью DummyClassifier

In [13]:
model_dummy_clf = DummyClassifier(strategy="most_frequent")
model_dummy_clf.fit(features_train, target_train)
predictions = model_dummy_clf.predict(features_test)
print("Accuracy модели DummyClassifier с параметром most_frequent:", accuracy_score(target_test, predictions))

Accuracy модели DummyClassifier с параметром most_frequent: 0.6936236391912908


**Вывод**
Модель решающего дерева дает выше Accuracy, чем ответы из DummyClassifier. Модель получилась адекватной.